# LL for an input Image

In [1]:
# imports
from importlib import reload
import os
#
import h5py 

import torch
from tqdm.auto import tqdm

from ulmo import ood
from ulmo.models import autoencoders, flows, ConditionalFlow

from ulmo import io as ulmo_io
from ulmo.preproc import utils as pp_utils
from ulmo import plotting

# Init

## Device

In [2]:
device = torch.device('cpu')

In [20]:
model_path = '/home/xavier/Projects/Oceanography/AI/OOD/2010_128x128/'
d_path = '/home/xavier/Projects/Oceanography/AI/OOD/'

# Load up the model

## DCAE

In [4]:
reload(autoencoders)
dcae = autoencoders.DCAE.from_file(os.path.join(model_path, 'autoencoder.pt'), image_shape=(1, 64, 64), latent_dim=512)

In [5]:
dcae

DCAE(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
  )
  (to_z): Lin

## Flow

### Instantiate

In [6]:
flow = ConditionalFlow(
    dim=512, 
    context_dim=None, 
    transform_type='autoregressive', 
    n_layers=10, 
    hidden_units=256,
    n_blocks=2, 
    dropout=0.2, 
    use_batch_norm=False, 
    tails='linear', 
    tail_bound=10, 
    n_bins=5,
    min_bin_height=1e-3, 
    min_bin_width=1e-3, 
    min_derivative=1e-3, 
    unconditional_transform=False, 
    encoder=None)

### Load

In [7]:
flow.load_state_dict(torch.load(os.path.join(model_path, 'flow.pt'), map_location=device))

<All keys matched successfully>

# Build the OOD

In [8]:
pae = ood.ProbabilisticAutoencoder(dcae, flow, 'tmp/', device=device)

In [9]:
pae.device

device(type='cpu')

# Give it a whirl on hdf5!

In [10]:
# Compute log probs of fields in new HDF5 file
pae.compute_log_probs(
    input_file=os.path.join(d_path, 'MODIS_2010_95clear_128x128_small_train_iterative.h5'),
    dataset='train',
    output_file='./sample_output.h5',
    scaler=None)

> /home/xavier/Projects/Oceanography/Python/ulmo/ulmo/ood/ood.py(393)compute_log_probs()
-> latents = scaler.transform(np.concatenate(latents))
(Pdb) latents
[array([[-2.00591335e+01,  2.27832317e+01, -5.70436430e+00, ...,
        -2.60435653e+00, -9.55053234e+00, -8.21272278e+00],
       [ 6.65678406e+01, -3.09173355e+01,  1.11708736e+01, ...,
         5.11626205e+01, -1.36297121e+01, -8.06563091e+00],
       [ 4.11085472e+01,  5.85056019e+00,  1.42332468e+01, ...,
        -2.90172691e+01,  2.62585106e+01, -5.30288544e+01],
       ...,
       [-1.20548859e+01, -2.81182218e+00,  4.12079096e+00, ...,
         1.36153269e+00, -1.17922325e+01, -1.20468845e+01],
       [-2.05650445e+21, -4.76759839e+21,  5.68748676e+21, ...,
         4.44566856e+19,  2.25754810e+21, -7.09470556e+20],
       [ 3.98377533e+01,  8.79060268e+00,  3.14937329e+00, ...,
         1.05573326e-01,  1.72325268e+01,  4.47029948e-01]], dtype=float32), array([[ 13.550908 , -16.485273 ,   1.8218529, ...,   2.0160747,
   

(Pdb) latents[0]
array([[-2.00591335e+01,  2.27832317e+01, -5.70436430e+00, ...,
        -2.60435653e+00, -9.55053234e+00, -8.21272278e+00],
       [ 6.65678406e+01, -3.09173355e+01,  1.11708736e+01, ...,
         5.11626205e+01, -1.36297121e+01, -8.06563091e+00],
       [ 4.11085472e+01,  5.85056019e+00,  1.42332468e+01, ...,
        -2.90172691e+01,  2.62585106e+01, -5.30288544e+01],
       ...,
       [-1.20548859e+01, -2.81182218e+00,  4.12079096e+00, ...,
         1.36153269e+00, -1.17922325e+01, -1.20468845e+01],
       [-2.05650445e+21, -4.76759839e+21,  5.68748676e+21, ...,
         4.44566856e+19,  2.25754810e+21, -7.09470556e+20],
       [ 3.98377533e+01,  8.79060268e+00,  3.14937329e+00, ...,
         1.05573326e-01,  1.72325268e+01,  4.47029948e-01]], dtype=float32)
(Pdb) latents[0].shape
(1024, 512)
(Pdb) len(latents)
40
(Pdb) tmp = np.concatenate(latents)
(Pdb) tmp.shape
(40000, 512)
(Pdb) exit


BdbQuit: 

----

# Try a single image

## Load and Pre-process a field

In [11]:
d_path = '/home/xavier/Projects/Oceanography/AI/ODD/'
filename = os.path.join(d_path, 'A2010059070500.L2_LAC_SST.nc')
# Load the image
sst, qual, latitude, longitude = ulmo_io.load_nc(filename, verbose=False)

# Generate the masks
masks = pp_utils.build_mask(sst, qual)

# Grab the field and mask
field_size = (128, 128)
row, col =  1209, 377

field = sst[row:row + field_size[0], col:col + field_size[1]]
mask = masks[row:row + field_size[0], col:col + field_size[1]]

print("This field has {:0.1f}% cloud coverage".format(100*mask.sum()/field.size))

# Pre-process
pp_field, mu = pp_utils.preproc_field(field, mask)

This field has 3.8% cloud coverage


## Get the Latent space

In [12]:
pp_field.resize(1,1,64,64)

In [13]:
dset = torch.utils.data.TensorDataset(torch.from_numpy(pp_field).float())

In [14]:
loader = torch.utils.data.DataLoader(
    dset, batch_size=1, shuffle=False, 
    drop_last=False, num_workers=16)

In [15]:
len(loader)

1

In [16]:
with torch.no_grad():
    latents = [dcae.encode(data[0].to(device)).detach().cpu().numpy()
             for data in tqdm(loader, total=len(loader), unit='batch', desc='Computing latents')]

In [19]:
latents[0].shape

(1, 512)

----

In [ ]:
modis_file = input_file=os.path.join(d_path, 'MODIS_2010_95clear_128x128_small_train_iterative.h5')

In [ ]:
h5f = h5py.File(modis_file, 'r')

In [ ]:
data = h5f['train'][0]

In [ ]:
data.shape

In [ ]:
type(data)